## Pytorch Linear Regression

Credit: https://towardsdatascience.com/linear-regression-with-pytorch-eb6dedead817

Input data will be in the following format for each restaurant business:
- food positive sentiment
- food negative sentiment
- service positive sentiment
- service negative sentiment
- location positive sentiment
- location negative sentiment
- cleanliness positive sentiment
- cleanliness negative sentiment
- price positive sentiment
- price negative sentiment

We want to predict the rating between 1 and 5 stars.

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import random_split

# REPLACE WITH PATH TO DATASET
df = pd.read_json('dataset')
data = df.to_numpy()
train, test = random_split(data, [0.8, 0.2], generator=torch.Generator().manual_seed(42))
# separate input attributes from target variable
x_train = train[:, 0:9]
y_train = train[:, 10]
x_test = test[:, 0:9]
y_test = test[:, 10]

In [3]:
! pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 MB 5.9 MB/s eta 0:00:0000:0100:01


In [1]:
from torch.autograd import Variable
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out

In [ ]:
inputDim = 10
outputDim = 1
learningRate = 0.2
epochs = 400

model = linearRegression(inputDim, outputDim)
##### For GPU #######
if torch.cuda.is_available():
    model.cuda()

In [ ]:
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

In [ ]:
for epoch in range(epochs):
    # Converting inputs and labels to Variable
    if torch.cuda.is_available():
        inputs = Variable(torch.from_numpy(x_train).cuda())
        labels = Variable(torch.from_numpy(y_train).cuda())
    else:
        inputs = Variable(torch.from_numpy(x_train))
        labels = Variable(torch.from_numpy(y_train))

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = model(inputs)

    # get loss for the predicted output
    loss = criterion(outputs, labels)
    print(loss)
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))

In [ ]:
import matplotlib.pyplot as plt

# Predict target variable using X test
with torch.no_grad():
    if torch.cuda.is_available():
        predicted = model(Variable(torch.from_numpy(x_test).cuda())).cpu().data.numpy()
    else:
        predicted = model(Variable(torch.from_numpy(x_test))).data.numpy()
    print(predicted)

plt.clf()
fig, axes = plt.subplots(5,2, figsize=(12,36))
axes = axes.flatten()

# plot model predictions with respect to each attribute
i = 0
for j in range(0,10):
    axes[i].plot(x_test[j], y_test, 'go', label='True data', alpha=0.5)
    axes[i].plot(x_test[j], predicted, '--', label='Predictions', alpha=0.5)
    i+=1
plt.legend(loc='best')
plt.show()